last updated 07 11 24

This notebook is to get the run times for each model on the highets and lowest Resolutions; to estimate an average run time.IG DICITONARY!



In [1]:
#!pip install ipython

In [2]:
### I want to load in a model
# and then test it! on blocked skyline images.


In [3]:
# imports
import torch

import torch.nn as nn
from torchvision.models import vgg16
#import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
from torchvision.models import vgg16
from torch.utils.data import DataLoader
#from torch.Utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import numpy as np
import cv2


from datetime import date
from tqdm import tqdm
import pprint
import collections
from IPython.display import clear_output
import time
import random
import cv2

import csv
import json
import pickle
import os

import wandb

import sys
sys.path.append('../.')
from functions import import_imagedata, ImageProcessor, label_oh_tf, IDSWDataSetLoader8
from fns4wandb import set_lossfn
from architectures import sevennet, smallnet1, smallnet2, smallnet3, PrintLayer
#from architectures import , smallnet3
from loop_fns import loop#, loop_batch, test_loop_batch
from plotting import learning_curve, accuracy_curve, plot_confusion
from modelManagment import get_lin_lay, choose_model2

import copy

from modelManagment import get_lin_lay, choose_model
from functions import get_data
from loop_fns import train_val_batch, test_loop_batch
from fileManagment import save2csv, save2json

#import torch.Utils.data.DataLoader as DataLoader

In [4]:
#p = torch.cuda.memory_summary(device, abbreviated=False)
#Pp = pprint.PrettyPrinter(indent=4)
#Pp.pprint(p)

In [5]:
# file paths
_save_location = r'/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/BlockSkyLine/4c/' #vgg16

data_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

gitHASH = '3e1975b1ac9d8c07dae424e08dacb2d2ce5f54f9'



load_in_model = False
dir_pkl = f"/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/schedulerRuns/4c/70E/"
#pkl_name = f"{model_name}_{resolution}_2024-11-19_{optimmy}_{int(config['start_epoch'])}E_{Var_WB_sched}_{resolution}_{str(lr)}_{str(wd_card)}_{seed}_CrossEntropy.pkl"


pkl_name_57 = "4c3l_[57, 18]_2025-01-08_Adam_70E_NoSched_run_[57, 18]_0.001_0_42_MSE.pkl"
pkl_name_113 ="4c3l_[113, 36]_2025-01-08_Adam_70E_NoSched_run_[113, 36]_0.001_0_42_MSE.pkl"

In [6]:
#wandb.login()


In [7]:
#!pip install datetime

d = date.today()
#print(str(d), type(str(d)))

452 144 5/452 *100 = 1%
226 72 5/226 *100 = 2%
113 36 5/113 *100 = 4% -- 2/113 *100= 1.7% ~ 2%
57 18 (56.5,) 5/57 *100 = 8% -- 2/57 *100 = 3.5% ~ 4%. 1/57 = 1.75%
29 9 (28.5,) 5/29 *100 = 17% -- 2/29 *100 = 6.89 ~ 7% 1/28 = 3.57 ~ 4%
15 5 (14.5, 4.5)
8 3 (7.5,2.5)
4, 2 (, 1.5)

In [8]:
# dictionaries                                                                                  * * * *   SETTINGS   * * * *

date = date.today()

model_card_vgg = {'name': 'vgg', 'model': 'vgg16',
                  'f_lin_lay':[200704,#200704,     #129024,#4096,  # (1x229376 and 25088x4096)  1x229376 and 25088x4096) 1x229376 and 25088x4096)
                             200704,      #(16x64512 and 129024x4096)    (16x200704 and 64512x4096)
                             14336,
                             3584,
                             768,
                             4096,
                             4096,
                            ],
                 'idx': 0,
                 'dropout':0.2}


model_card_7c3l = {'name': '7c3l', 'model': '7c3l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[248832,    # 452 144 # p5
                            59904,      # 226 72 # p5
                            11264,      # 113 36 # p2
                            1536,       # 57 18 # p1
                            172032,           # 29 9
                            172032,          # 15 5
                            172032,         # 8 3
                              ], 
                   'idx': 1,
                  'dropout':0.2}



model_card_4c3l = {'name': '4c3l', 'model': '4c3l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[539904,# 1055232,#539904,    # 452 144 # p5  (64x539904 and 1055232x100)
                             141056, #141056,    # 226 72 # p5   64x141056 and 267264x100)
                             34560, #35840,     # 113 36 # p2   (64x35840 and 304640x100)
                             9984,      # 57 18 # p1 
                             2304,      # 29 9
                             512,       # 15 5
                             256],      # 8 3
                  'idx': 2,
                  'dropout':0.2}      

model_card_3c2l = {'name': '3c2l', 'model': '3c2l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[1069888,    # 452 144 # p5
                             274688,     #226 72 # p5
                             68096,      # 113 36 # p2
                             17280,      # 57 18 # p1
                             3840,       # 29 9
                             960,        # 15 5
                             256],
                  'idx': 3,
                  'dropout':0.2}       # 8 3

model_card_2c2l = {'name': '2c2l', 'model': '2c2l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[1055232 , #1032192,# 16883712,#33767424,    # 452 144 # p5 # (1x33767424 and 1055232x100) (1x5276160 and 15828480x100) 1x33767424 and 5276160x100)
                             267264,     #226 72 # p5                   (1x1032192 and 64512x100)
                             64512,#   1032192,#64512,      # 113 36 # p2    ### (16x1055232 and 1032192x100) ###  16x1055232 and 1032192x100)
                             15552,      # 57 18 # p1
                             3072,       # 29 9
                             640,        # 15 5
                             128],
                  'idx': 4,
                  'dropout':0.1}       # 8 3

resolution_card_452144 = {'resolution':[452,144], 'padding':5, 'index':0}
resolution_card_22672 = {'resolution':[226,72], 'padding':5, 'index':1}
resolution_card_11336 = {'resolution':[113,36], 'padding':2, 'index':2}
resolution_card_5715 = {'resolution':[57,18], 'padding':1, 'index':3}

resolution_card_299 = {'resolution':[29,9], 'padding':0, 'index':4} # 
resolution_card_155 = {'resolution':[15,5], 'padding':0, 'index':5}
resolution_card_83 = {'resolution':[8,3], 'padding':0, 'index':6}



resolution_cards = [resolution_card_11336, resolution_card_5715]
                      #[resolution_card_11336, resolution_card_5715,,resolution_card_155
#resolution_card_299, resolution_card_155, resolution_card_83]#]#resolution_card_452144, 
#resolution_cards = [resolution_card_11336]

#learning_rate_cards = [5e-5, 6e-5, 8e-5]
#learning_rate_cards = [8.21592E-05, 6.62E-05, 6.01E-05, 5.97E-05]
#learning_rate_cards=  [1e-3] #[0.1,0.01, 1e-3,1e-4, 1e-5]#, 6e-5, 7e-5, 8e-5]
#wd_cards = [4e-5, 5e-5, 3.00E-05, 2.00E-05]
#wd_cards =[0]
#scheduler_cards = [0]#, 0.1, 0.2]

seeds = [8,2,4, 42]#,2,3] # 4, 5,6

#model_cards =[model_card_vgg, model_card_7c3l, model_card_4c3l, model_card_3c2l, model_card_2c2l]
model_cards =[model_card_4c3l]

#loss_fn_cards = ['MSE'] #,'CrossEntropy' 
                        
config = dict({'parameters': 'parameters for big loop run'})
config.update({'model_cards':model_cards})
config.update({'resolution_cards':resolution_cards})
#config.update({'learning_rate_cards':learning_rate_cards})
#config.update({'wd_cards':wd_cards})
#config.update({'scheduler_cards':scheduler_cards})
config.update({'seeds':seeds})
#config.update({'loss_fn_cards': loss_fn_cards})


config.update({'batch_size': 64})
config.update({'epochs': 60}) #60

#print(model_card_vgg)
#print('')
#Pp.pprint(Config) # dictionary of dictionaries of lists and lists of dictionaries

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


def _go(config=None):
    if len(gitHASH) <1:
        print("YOU FORGET THE GIT HASH")
        return
    else:
        #print('Git Hash registered')
        pass

    start = time.process_time()
        
    for model_idx, model_card in enumerate(config['model_cards']):

        model_name = model_card['model']
        model_index = model_card['idx']
        dropout = model_card['dropout'] 
        for res_idx, resolution_card in enumerate(config['resolution_cards']):
            resolution = resolution_card['resolution']
            pad = resolution_card['padding']
            lin_lay = get_lin_lay(model_card, resolution)
            print('lin lay', lin_lay)
        
            for seed_idx, seed in enumerate(config['seeds']):
                seed = seed
                
                torch.cuda.empty_cache()
                config['batch_size']

                print('Model: ', str(model_name), f" idx: {model_idx} / {len(config['model_cards'])}")
                print('resolution: ', str(resolution), f" idx: {res_idx} / {len(config['resolution_cards'])}")
                print('seed: ', str(seed), f" idx: {seed_idx} / {len(config['seeds'])}")
                print('Batch size: ', config['batch_size'])
                

                run_start_time = time.process_time()
                print('start time: ',run_start_time)

                print(time.process_time() - start)

                IP = ImageProcessor(device)


                save_dict = {'Run' : f"{model_name}_{resolution}_{date}_BlockSkyLine_test",
                             'Current_Epoch': 0,
                             'save_location' : _save_location}

                model = choose_model2(model_name, lin_lay, dropout).to(device)
                if load_in_model:
                    if resolution == "[57, 18]":
                        pkl_name = pkl_name_57

                    elif resolution == "[113, 32]":
                        pkl_name_113
                    else:
                        print("Resolution not recognised!")
                    with open(dir_pkl+pkl_name, 'rb') as f:
                        model_pkl = pickle.load(f)
                    #model_pkl = pickle.load(dir_pkl+pkl_name)
                    model.load_state_dict(model_pkl['model.state_dict'])
                # this is where is load in a pretrained model
                #print("Before model init - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                #model = smallnet3(in_chan=3, f_lin_lay=int(lin_lay), l_lin_lay=11, ks= (3,5), dropout= dropout).to(device)

                #print('4')
                #!nvidia-smi

                #print("After model init, Before data loading - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)

                x_train, y_train, x_val, y_val, x_test, y_test = get_data(seed,  file_path=data_path)
                av_lum = IP.new_luminance(x_train)
                print("having just got the data: img:", len(x_test), "lab:", len(y_test))
                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                
                #train_ds = IDSWDataSetLoader8(x_train, y_train, resolution,pad,av_lum,model_name, device)# av_lum, res,pad,
                #train = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2

                #  img_path, labels, av_lum, transform=None, res = (452, 144), vgg=False
                test_ds = IDSWDataSetLoader8(x_test, y_test,av_lum, res=resolution)
                test = DataLoader(test_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2

                #loss_fn = set_lossfn(loss)

                loop_run_name = f"TEST_SKYLINE_OBTRUSION_{save_dict['Run']}_{resolution}_{seed}"
                #                                                   (model,data, loss_fn, batch_size, device
                test_acc,test_predict_list, y_test_ = test_loop_batch(model,test, loss_fn=None, batch_size=config['batch_size'], device=device) #model, model_name, X, Y, res, pad, loss_fn, device, num_classes=11
                print("in go, after the test_loop_batch, type len  pred:", type(test_predict_list), len(test_predict_list), " y_test:", type(y_test), len(y_test))
                #print(test_predict_list)

                print(' \n test Acc: ', test_acc)
                
                save_dict.update({'test_acc': test_acc})
                save_dict.update({'test_predict': test_predict_list})
                save_dict.update({'test_labels': list(y_test)})
                #save_dict.update({'test_loss':test_loss})

                #test_predict_list=[pred.cpu() for pred in test_predict_list]
                plot_confusion(predictions= test_predict_list, actual= y_test_, title = "Test Confusion matrix", run_name = loop_run_name,save_location =save_dict['save_location'])
                
                #saving
                diction = {}
                d = date.today()
                d=str(d)
                diction.update({'Date':d})
                diction.update({'gitHASH':str(gitHASH)})
                diction.update({'model_name': str(model_name)})
                diction.update({'seed': str(seed)})
                diction.update({'resolution': str(resolution)})
                diction.update({'pad': int(pad)})
                diction.update({'lin_lay': int(lin_lay)})
                diction.update({'run time': (time.process_time() - run_start_time)})
                diction.update(save_dict)
                
                save_location = save_dict['save_location']
                title = save_dict['Run']
                save2json(diction, loop_run_name, save_location)
                save2csv(diction, title, save_location)

                diction['model.state_dict'] = model.state_dict() #to('cpu').

                with open(f"{save_location}{loop_run_name}.pkl", 'wb+') as f:
                    torch.save(diction, f)
                
                clear_output()
                
                print(f' \n END {model_name} {resolution} Run Time: ',time.process_time() - run_start_time)
                #!nvidia-smi
                torch.cuda.empty_cache()
    print('Final Run time: ',time.process_time() - start)

In [10]:
_go(config)

lin lay 34560
Model:  4c3l  idx: 0 / 1
resolution:  [113, 36]  idx: 0 / 2
seed:  8  idx: 0 / 4
Batch size:  64
start time:  12.431908144
5.5888000000337e-05
/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/
having just got the data: img: 1030 lab: 1030
start of get_item 9
just before label_oh_tf 9
label_oh_tf init
in label_oh_tf. label: 9
label_oh_tf. lab post intting 9
label_oh_tf. one hotted: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
as a tensor. lab: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=torch.float64)
end lebel_oh_tf, label: <class 'torch.Tensor'> tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
in get_item, label: <class 'tuple'> (tensor([[[0.6902, 0.7216, 0.7647,  ..., 0.6431, 0.6706, 0.6941],
         [0.8196, 0.7490, 0.6627,  ..., 0.9176, 0.8588, 0.7961],
         [0.9882, 0.9961, 0.9961,  ..., 0.9333, 0.9725, 0.9882],
         ...,
         [0.6078, 0.6078, 0.6078,  ..., 0.2353, 0.2392, 0.2235],
         [0.6078, 0.6078, 0.6078,  ..., 0.2275, 0.2118, 0.2

/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
in get_item, label: <class 'tuple'> (tensor([[[0.7294, 0.7412, 0.7529,  ..., 0.7098, 0.7216, 0.7294],
         [0.8627, 0.8392, 0.8314,  ..., 0.8824, 0.8745, 0.8510],
         [0.7608, 0.6980, 0.6235,  ..., 0.6902, 0.7647, 0.7373],
         ...,
         [0.1647, 0.1608, 1.0000,  ..., 0.1608, 0.1686, 0.1843],
         [0.1569, 0.1647, 1.0000,  ..., 0.1725, 0.1804, 0.1412],
         [0.0824, 0.0824, 1.0000,  ..., 0.0980, 0.0980, 0.0745]],

        [[0.7843, 0.7961, 0.8000,  ..., 0.7608, 0.7725, 0.7804],
         [0.9176, 0.8902, 0.8745,  ..., 0.9412, 0.9333, 0.9059],
         [0.8392, 0.7647, 0.6667,  ..., 0.7686, 0.8431, 0.8118],
         ...,
         [0.1490, 0.1333, 0.6078,  ..., 0.1333, 0.1529, 0.1490],
         [0.1412, 0.1490, 0.6078,  ..., 0.1725, 0.1647, 0.1216],
         [0.0784, 0.0784, 0.6078,  ..., 0.0902, 0.0941, 0.0784]],

        [[0.6275, 0.6431, 0.6510,  ..., 0.6196, 0.6196, 0.6275],
         [0.7490, 0.7373, 0.7216,

KeyboardInterrupt: 

In [ ]:
# 12.58 GiB. GPU 0 has a total capacty of 23.65 GiB of which 8.04 GiB is free
23.65-8.04


pred torch.Size([11])

lab  torch.Size([5, 11])